In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Merge, Reshape, Dropout
from keras.layers.embeddings import Embedding
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [3]:
np.random.seed(10)
tf.set_random_seed(15)

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6257724299872109230
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17248580277577767522
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 16585632572182645017
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1838350336
locality {
  bus_id: 1
}
incarnation: 16909122603326011277
physical_device_desc: "device: 0, name: GeForce 830M, pci bus id: 0000:08:00.0, compute capability: 5.0"
]


In [5]:
def gini(actual, pred):
    assert (len(actual) == len(pred))
    all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)
    all = all[np.lexsort((all[:, 2], -1 * all[:, 1]))]
    totalLosses = all[:, 0].sum()
    giniSum = all[:, 0].cumsum().sum() / totalLosses

    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)


def gini_normalized(actual, pred):
    return gini(actual, pred) / gini(actual, actual)

#### Load data 

In [12]:
data = pd.read_csv("../data/train.csv")
calc_cols = data.columns[data.columns.str.startswith('ps_calc_')]
data = data.drop(["id"] + calc_cols.tolist(), axis=1)
X, y = data.loc[:, data.columns != "target"], data.target

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

/usr/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [14]:
y_train.index

Int64Index([391389, 518243, 136933, 432345, 127021, 141310,  72757, 480572,
            177798,  23316,
            ...
            175203,  87498, 521430, 137337,  54886, 110268, 259178, 365838,
            131932, 121958],
           dtype='int64', length=416648)

#### Pre-Processing 

In [8]:
cat_cols = X.columns[X.columns.str.endswith("_cat")]
embed_cols = [c for c in cat_cols if (X[c].nunique(dropna=False) > 2)]
not_embed_cols = [c for c in X.columns if c not in embed_cols]

In [9]:
for c in embed_cols:
    le = LabelEncoder()
    le.fit(X[c])
    X_train.loc[:, c] = le.transform(X_train[c])
    X_test.loc[:, c] = le.transform(X_test[c])

/usr/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [10]:
def get_input_list(X, embed_cols, not_embed_cols):
    input_list = []
    for c in embed_cols:
        input_list.append(X.loc[:, c].values)
    
    input_list.append(X.loc[:, not_embed_cols].values)
    return input_list

#### Model 

In [11]:
def build_model():
    models = []
    
    model_ps_ind_02_cat = Sequential()
    model_ps_ind_02_cat.add(Embedding(5, 3, input_length=1))
    model_ps_ind_02_cat.add(Reshape(target_shape=(3,)))
    models.append(model_ps_ind_02_cat)
    
    model_ps_ind_04_cat = Sequential()
    model_ps_ind_04_cat.add(Embedding(3, 2, input_length=1))
    model_ps_ind_04_cat.add(Reshape(target_shape=(2,)))
    models.append(model_ps_ind_04_cat)
    
    model_ps_ind_05_cat = Sequential()
    model_ps_ind_05_cat.add(Embedding(8, 5, input_length=1))
    model_ps_ind_05_cat.add(Reshape(target_shape=(5,)))
    models.append(model_ps_ind_05_cat)
    
    model_ps_car_01_cat = Sequential()
    model_ps_car_01_cat.add(Embedding(13, 7, input_length=1))
    model_ps_car_01_cat.add(Reshape(target_shape=(7,)))
    models.append(model_ps_car_01_cat)
    
    model_ps_car_02_cat = Sequential()
    model_ps_car_02_cat.add(Embedding(3, 2, input_length=1))
    model_ps_car_02_cat.add(Reshape(target_shape=(2,)))
    models.append(model_ps_car_02_cat)
    
    model_ps_car_03_cat = Sequential()
    model_ps_car_03_cat.add(Embedding(3, 2, input_length=1))
    model_ps_car_03_cat.add(Reshape(target_shape=(2,)))
    models.append(model_ps_car_03_cat)
    
    model_ps_car_04_cat = Sequential()
    model_ps_car_04_cat.add(Embedding(10, 5, input_length=1))
    model_ps_car_04_cat.add(Reshape(target_shape=(5,)))
    models.append(model_ps_car_04_cat)
    
    model_ps_car_05_cat = Sequential()
    model_ps_car_05_cat.add(Embedding(3, 2, input_length=1))
    model_ps_car_05_cat.add(Reshape(target_shape=(2,)))
    models.append(model_ps_car_05_cat)
    
    model_ps_car_06_cat = Sequential()
    model_ps_car_06_cat.add(Embedding(18, 8, input_length=1))
    model_ps_car_06_cat.add(Reshape(target_shape=(8,)))
    models.append(model_ps_car_06_cat)
    
    model_ps_car_07_cat = Sequential()
    model_ps_car_07_cat.add(Embedding(3, 2, input_length=1))
    model_ps_car_07_cat.add(Reshape(target_shape=(2,)))
    models.append(model_ps_car_07_cat)
    
    model_ps_car_09_cat = Sequential()
    model_ps_car_09_cat.add(Embedding(6, 3, input_length=1))
    model_ps_car_09_cat.add(Reshape(target_shape=(3,)))
    models.append(model_ps_car_09_cat)
    
    model_ps_car_10_cat = Sequential()
    model_ps_car_10_cat.add(Embedding(3, 2, input_length=1))
    model_ps_car_10_cat.add(Reshape(target_shape=(2,)))
    models.append(model_ps_car_10_cat)
    
    model_ps_car_11_cat = Sequential()
    model_ps_car_11_cat.add(Embedding(104, 10, input_length=1))
    model_ps_car_11_cat.add(Reshape(target_shape=(10,)))
    models.append(model_ps_car_11_cat)
    
    model_rest = Sequential()
    model_rest.add(Dense(16, input_dim=24))
    models.append(model_rest)

    model = Sequential()
    model.add(Merge(models, mode='concat'))
    model.add(Dense(80))
    model.add(Activation('relu'))
    model.add(Dropout(.35))
    model.add(Dense(20))
    model.add(Activation('relu'))
    model.add(Dropout(.15))
    model.add(Dense(10))
    model.add(Activation('relu'))
    model.add(Dropout(.15))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam')
    
    return model

In [16]:
n_splits = 8
n_runs = 3
n_epochs = 30
batch_size = 4096

cv_ginis = []
full_val_preds = np.zeros(np.shape(X_train)[0])
y_preds = np.zeros((np.shape(X_test)[0], n_splits))

proc_X_test = get_input_list(X_test, embed_cols, not_embed_cols)
proc_X_train = get_input_list(X_train, embed_cols, not_embed_cols)
kfold = StratifiedKFold(n_splits=n_splits, random_state=231, shuffle=True)

In [17]:
for k, (idx, val_idx) in enumerate(kfold.split(X_train, y_train)):
    X_fold, X_val = X_train.iloc[idx].copy(), X_train.iloc[val_idx].copy()
    y_fold, y_val = y_train.iloc[idx], y_train.iloc[val_idx]
    
    # Upsample
    #  https://www.kaggle.com/ogrellier/xgb-classifier-upsampling-lb-0-283
    pos = (y_fold == 1)
    X_fold = pd.concat([X_fold, X_fold.loc[pos, :]], axis=0)
    y_fold = pd.concat([y_fold, y_fold.loc[pos]], axis=0)
    
    # Shuffle
    shuffled_idx = np.arange(len(X_fold))
    np.random.shuffle(shuffled_idx)
    X_fold = X_fold.iloc[shuffled_idx]
    y_fold = y_fold.iloc[shuffled_idx]
    
    # Preprocessing
    proc_X_fold = get_input_list(X_fold, embed_cols, not_embed_cols)
    proc_X_val = get_input_list(X_val, embed_cols, not_embed_cols)
    
    # Run several times for this fold and average the predictions
    val_preds = 0
    for r in range(n_runs):
        print(r)
        model = build_model()
        model.fit(proc_X_fold, y_fold.values, epochs=n_epochs, batch_size=batch_size, verbose=True)
        val_preds += model.predict(proc_X_val)[:, 0] / n_runs
        
        y_preds[:, k] += model.predict(proc_X_test)[:, 0] / n_runs
        y_preds_train[:, k] += model.predict(proc_X_train)[:, 0] / n_runs
    full_val_preds[val_idx] += val_preds
    
    # Save results on the validation set
    cv_gini = gini_normalized(y_val.values, val_preds)
    cv_ginis.append(cv_gini)
    print('\nFold %i prediction cv gini: %.5f\n' %(k, cv_gini))
    
print('Mean out of fold gini: %.5f' % np.mean(cv_ginis))
print('Full validation gini: %.5f' % gini_normalized(y_train.values, full_val_preds))

0


/usr/lib/python3.6/site-packages/ipykernel/__main__.py:74: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Epoch 1/30
377786/377786 [==============================] - 5s 13us/step - loss: 0.3920
Epoch 2/30
377786/377786 [==============================] - 2s 6us/step - loss: 0.2786
Epoch 3/30
377786/377786 [==============================] - 2s 6us/step - loss: 0.2674
Epoch 4/30
377786/377786 [==============================] - 2s 6us/step - loss: 0.2643
Epoch 5/30
377786/377786 [==============================] - 2s 6us/step - loss: 0.2630
Epoch 6/30
377786/377786 [==============================] - 2s 6us/step - loss: 0.2619
Epoch 7/30
377786/377786 [==============================] - 2s 6us/step - loss: 0.2609
Epoch 8/30
377786/377786 [==============================] - 2s 6us/step - loss: 0.2604
Epoch 9/30
377786/377786 [==============================] - 2s 6us/step - loss: 0.2597
Epoch 10/30
377786/377786 [==============================] - 2s 6us/step - loss: 0.2590
Epoch 11/30
377786/377786 [==============================] - 2s 6us/step - loss: 0.2587
Epoch 12/30
377786/377786 [=============

377786/377786 [==============================] - 2s 6us/step - loss: 0.2585
Epoch 7/30
377786/377786 [==============================] - 2s 6us/step - loss: 0.2575
Epoch 8/30
377786/377786 [==============================] - 2s 6us/step - loss: 0.2570
Epoch 9/30
377786/377786 [==============================] - 2s 6us/step - loss: 0.2558
Epoch 10/30
377786/377786 [==============================] - 2s 6us/step - loss: 0.2555
Epoch 11/30
377786/377786 [==============================] - 2s 6us/step - loss: 0.2547
Epoch 12/30
377786/377786 [==============================] - 2s 6us/step - loss: 0.2544
Epoch 13/30
377786/377786 [==============================] - 2s 6us/step - loss: 0.2538
Epoch 14/30
377786/377786 [==============================] - 2s 6us/step - loss: 0.2533
Epoch 15/30
377786/377786 [==============================] - 2s 6us/step - loss: 0.2529
Epoch 16/30
377786/377786 [==============================] - 2s 6us/step - loss: 0.2522
Epoch 17/30
377786/377786 [====================

377787/377787 [==============================] - 2s 6us/step - loss: 0.2579
Epoch 10/30
377787/377787 [==============================] - 2s 6us/step - loss: 0.2576
Epoch 11/30
377787/377787 [==============================] - 2s 6us/step - loss: 0.2567
Epoch 12/30
377787/377787 [==============================] - 2s 6us/step - loss: 0.2565
Epoch 13/30
377787/377787 [==============================] - 2s 6us/step - loss: 0.2564
Epoch 14/30
377787/377787 [==============================] - 2s 6us/step - loss: 0.2561
Epoch 15/30
377787/377787 [==============================] - 2s 6us/step - loss: 0.2550
Epoch 16/30
377787/377787 [==============================] - 2s 6us/step - loss: 0.2553
Epoch 17/30
377787/377787 [==============================] - 2s 6us/step - loss: 0.2545
Epoch 18/30
377787/377787 [==============================] - 2s 6us/step - loss: 0.2538
Epoch 19/30
377787/377787 [==============================] - 2s 6us/step - loss: 0.2535
Epoch 20/30
377787/377787 [=================

377787/377787 [==============================] - 2s 6us/step - loss: 0.2516
Epoch 13/30
377787/377787 [==============================] - 2s 6us/step - loss: 0.2512
Epoch 14/30
377787/377787 [==============================] - 2s 6us/step - loss: 0.2505
Epoch 15/30
377787/377787 [==============================] - 2s 6us/step - loss: 0.2503
Epoch 16/30
377787/377787 [==============================] - 2s 6us/step - loss: 0.2503
Epoch 17/30
377787/377787 [==============================] - 2s 6us/step - loss: 0.2496
Epoch 18/30
377787/377787 [==============================] - 2s 6us/step - loss: 0.2491
Epoch 19/30
377787/377787 [==============================] - 2s 6us/step - loss: 0.2491
Epoch 20/30
377787/377787 [==============================] - 2s 6us/step - loss: 0.2489
Epoch 21/30
377787/377787 [==============================] - 2s 6us/step - loss: 0.2485
Epoch 22/30
377787/377787 [==============================] - 2s 6us/step - loss: 0.2479
Epoch 23/30
377787/377787 [=================

377789/377789 [==============================] - 2s 6us/step - loss: 0.2512
Epoch 16/30
377789/377789 [==============================] - 2s 6us/step - loss: 0.2511
Epoch 17/30
377789/377789 [==============================] - 2s 6us/step - loss: 0.2504
Epoch 18/30
377789/377789 [==============================] - 2s 6us/step - loss: 0.2502
Epoch 19/30
377789/377789 [==============================] - 2s 6us/step - loss: 0.2499
Epoch 20/30
377789/377789 [==============================] - 2s 6us/step - loss: 0.2497
Epoch 21/30
377789/377789 [==============================] - 2s 6us/step - loss: 0.2487
Epoch 22/30
377789/377789 [==============================] - 2s 6us/step - loss: 0.2486
Epoch 23/30
377789/377789 [==============================] - 2s 6us/step - loss: 0.2482
Epoch 24/30
377789/377789 [==============================] - 2s 6us/step - loss: 0.2481
Epoch 25/30
377789/377789 [==============================] - 2s 6us/step - loss: 0.2476
Epoch 26/30
377789/377789 [=================

Epoch 18/30
377789/377789 [==============================] - 2s 6us/step - loss: 0.2501
Epoch 19/30
377789/377789 [==============================] - 2s 6us/step - loss: 0.2502
Epoch 20/30
377789/377789 [==============================] - 2s 6us/step - loss: 0.2497
Epoch 21/30
377789/377789 [==============================] - 2s 6us/step - loss: 0.2495
Epoch 22/30
377789/377789 [==============================] - 2s 6us/step - loss: 0.2490
Epoch 23/30
377789/377789 [==============================] - 2s 6us/step - loss: 0.2488
Epoch 24/30
377789/377789 [==============================] - 2s 6us/step - loss: 0.2484
Epoch 25/30
377789/377789 [==============================] - 2s 6us/step - loss: 0.2481
Epoch 26/30
377789/377789 [==============================] - 2s 6us/step - loss: 0.2477
Epoch 27/30
377789/377789 [==============================] - 2s 6us/step - loss: 0.2476
Epoch 28/30
377789/377789 [==============================] - 2s 6us/step - loss: 0.2474
Epoch 29/30
377789/377789 [=====

In [18]:
y_pred_final = np.mean(y_preds, axis=1)
y_pred_train_final = np.mean(y_preds_train, axis=1)
pd.DataFrame({"test": y_pred_final}).to_csv("test_results_nn.csv", index=False)
pd.DataFrame({"train": y_pred_train_final}).to_csv("train_results_nn.csv", index=False)
gini_normalized(y_test.values, y_pred_final)

0.27561994764745495